# Comparative Stock & Revenue Dashboard

This notebook extracts historical stock data and revenue data for Tesla and GameStop using yfinance and web scraping, and then visualizes the data using interactive Plotly dashboards.

## Table of Contents

1. **Question 1:** Extract Tesla Stock Data using yfinance
2. **Question 2:** Extract Tesla Revenue Data via Web Scraping
3. **Question 3:** Extract GameStop Stock Data using yfinance
4. **Question 4:** Extract GameStop Revenue Data via Web Scraping
5. **Question 5:** Plot Tesla Stock & Revenue Dashboard
6. **Question 6:** Plot GameStop Stock & Revenue Dashboard

In [ ]:
# Imports and Setup
!pip install yfinance bs4 html5lib lxml pandas matplotlib plotly

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Question 1: Extract Tesla Stock Data using yfinance

This cell creates a Tesla ticker object, extracts historical stock data with a maximum period, resets the index, and displays the first five rows.

In [ ]:
# Question 1

tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
print("Question 1: Tesla Stock Data - First 5 rows:")
tesla_data.head()

## Question 2: Extract Tesla Revenue Data via Web Scraping

This cell downloads the Tesla revenue webpage, extracts the revenue table using pandas.read_html, cleans the data, and displays the last five rows. The table is selected by checking if its content contains the word "Quarterly" so that quarterly (not annual) data is obtained.

In [ ]:
# Question 2
from io import StringIO

url_tesla_revenue = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data_tesla = requests.get(url_tesla_revenue).text

# Extract all tables from the HTML
tesla_revenue_list = pd.read_html(StringIO(html_data_tesla))

# Select the table that contains "Quarterly" in its content
tesla_revenue = None
for table in tesla_revenue_list:
    if "Quarterly" in table.to_string():
        tesla_revenue = table
        break
if tesla_revenue is None:
    tesla_revenue = tesla_revenue_list[0]

print("Columns in Tesla revenue table:", tesla_revenue.columns.tolist())

# Rename columns to "Date" and "Revenue"
tesla_revenue.columns = ["Date", "Revenue"]

# Clean the Revenue column: remove "$" and commas, then convert to numeric
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(r",|\$", "", regex=True)
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors="coerce")

print("Question 2: Tesla Revenue Data - Last 5 rows:")
tesla_revenue.tail()

## Question 3: Extract GameStop Stock Data using yfinance

This cell creates a GameStop ticker object, extracts historical stock data with a maximum period, resets the index, and displays the first five rows.

In [ ]:
# Question 3

gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
print("Question 3: GameStop Stock Data - First 5 rows:")
gme_data.head()

## Question 4: Extract GameStop Revenue Data via Web Scraping

This cell downloads the GameStop revenue webpage (from stock.html), selects the table that contains quarterly revenue data (by searching for "Quarterly" in its content), cleans it, and displays the last five rows.

In [ ]:
# Question 4
url_gme_revenue = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data_gme = requests.get(url_gme_revenue).text

# Extract all tables from the HTML
gme_revenue_list = pd.read_html(StringIO(html_data_gme))

# Select the table that contains "Quarterly" in its content
gme_revenue = None
for table in gme_revenue_list:
    if "Quarterly" in table.to_string():
        gme_revenue = table
        break
if gme_revenue is None:
    gme_revenue = gme_revenue_list[0]

print("Columns in GameStop revenue table:", gme_revenue.columns.tolist())

# Rename columns to "Date" and "Revenue"
gme_revenue.columns = ["Date", "Revenue"]

# Clean the Revenue column: remove "$" and commas, then convert to numeric
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(r",|\$", "", regex=True)
gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"], errors="coerce")

print("Question 4: GameStop Revenue Data - Last 5 rows:")
gme_revenue.tail()

## Define the make_graph Function

This function creates a dashboard with two subplots using Plotly: one for the historical stock closing price and one for the historical revenue data. It converts the "Date" columns to datetime (ensuring tz-naive) and filters the data up to specific dates before plotting.

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def remove_timezone(series):
    # Convertir la serie a datetime
    dt_series = pd.to_datetime(series, errors="coerce")
    # Si la serie es tz-aware, convertir a tz-naive; de lo contrario, forzar dtype
    if dt_series.dt.tz is not None:
        return dt_series.dt.tz_convert(None).astype("datetime64[ns]")
    else:
        return dt_series.astype("datetime64[ns]")

def make_graph(stock_data, revenue_data, stock):
    # Trabajar sobre copias para evitar modificar el original
    stock_data = stock_data.copy()
    revenue_data = revenue_data.copy()
    
    # Convertir las columnas "Date" a datetime tz-naive
    stock_data.loc[:, "Date"] = remove_timezone(stock_data["Date"])
    revenue_data.loc[:, "Date"] = remove_timezone(revenue_data["Date"])
    
    # Filtrar datos hasta las fechas indicadas
    stock_data_specific = stock_data[stock_data.Date <= pd.to_datetime("2021-06-14")]
    revenue_data_specific = revenue_data[revenue_data.Date <= pd.to_datetime("2021-04-30")]
    
    print("Number of revenue data rows after filtering:", len(revenue_data_specific))
    
    # Crear subgráficas
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        subplot_titles=("Historical Share Price", "Historical Revenue"),
                        vertical_spacing=0.3)
    
    # Traza para el precio de la acción
    fig.add_trace(go.Scatter(
        x=stock_data_specific.Date,
        y=stock_data_specific.Close.astype("float"),
        name="Share Price"
    ), row=1, col=1)
    
    # Traza para los ingresos
    fig.add_trace(go.Scatter(
        x=revenue_data_specific.Date,
        y=revenue_data_specific.Revenue.astype("float"),
        name="Revenue"
    ), row=2, col=1)
    
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue (USD Millions)", row=2, col=1)
    
    fig.update_layout(
        height=900,
        title=stock,
        showlegend=False,
        xaxis_rangeslider_visible=True
    )
    
    fig.show()


## Question 5: Plot Tesla Stock & Revenue Dashboard

This cell calls the `make_graph` function to display the Tesla dashboard graph with the historical stock price and revenue data.

In [ ]:
# Question 5
make_graph(tesla_data, tesla_revenue, "Tesla")

## Question 6: Plot GameStop Stock & Revenue Dashboard

This cell calls the `make_graph` function to display the GameStop dashboard graph with the historical stock price and revenue data.

In [ ]:
# Question 6
make_graph(gme_data, gme_revenue, "GameStop")